In [19]:
#! pip install scikit-learn

In [20]:
import requests
import json
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import gradio as gr
import pandas as pd

In [21]:
# API Credentials (Replace with your credentials)
CLIENT_ID = "8d3314772d90b39ef86aeed6ff9868aa"
API_URL = "https://api.myanimelist.net/v2/anime"
Fields = "id,title,alternative_titles,synopsis,genres,mean,rank,popularity,media_type,status"

In [22]:
def get_anime_data(anime_id):
    headers = {"X-MAL-Client-ID": CLIENT_ID}
    params = {"fields": Fields}
    response = requests.get(f"{API_URL}/{anime_id}", headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

In [23]:
def fetch_anime_list(offset=0):
    headers = {"X-MAL-Client-ID": CLIENT_ID}
    params = {"ranking_type": "all", "limit": 100, "offset": offset, "fields": Fields}
    response = requests.get(f"{API_URL}/ranking", headers=headers, params=params)
    if response.status_code == 200:
        return response.json().get("data", [])
    else:
        print(f"Error {response.status_code}: {response.text}")
        return []

In [24]:
# Extract base title (removing "Season X", "Movie", "Special" identifiers)
def combine_copies(new_data):   
    def extract_base_title(title):
            title = title.split(': ')[0].split(' - ')[0].strip()
            base_title = re.sub(r'\s*(Season|Movie|Special|OVA|Part|Final|1st|2nd|3rd|4th|5th|6th|7th|8th|9th|10th)\s*\d*', '', title, flags=re.IGNORECASE).strip()
            return base_title

    new_data["base_title"] = new_data['title'].apply(extract_base_title)
    
    # Group by base title and aggregate related anime
    grouped_data = new_data.groupby("base_title").agg({
        "id": lambda x: list(x.reset_index(drop=True)),  # Collect IDs
        "title": lambda x: list(x.reset_index(drop=True)),  # Collect all versions of the title
        "synopsis": lambda x: " ".join(set(x.reset_index(drop=True))),  # Merge unique synopses
        "genres": lambda x: ", ".join(set(", ".join(x.reset_index(drop=True)).split(", "))),  # Unique genres
        "mean": "mean",  # Compute the average score
        "media_type": lambda x: list(set(x.reset_index(drop=True))),  # Store unique media types
        "status": lambda x: list(set(x.reset_index(drop=True)))  # Store unique statuses
    }).reset_index()

    grouped_data.drop(columns = 'title', inplace=True)

    # Rename base_title to title for final output
    grouped_data.rename(columns={"base_title": "title"}, inplace=True)
    
    return grouped_data

In [25]:
def preprocess_data(anime_list, existing_df):
    # Create a DataFrame from the MAL API response
    new_data = pd.DataFrame([
        {
            "id": anime["node"]["id"],  # Unique MAL ID
            "title": anime["node"]["title"],  # Anime title
            "alternative_titles": anime["node"]["title"],  # Anime title
            "synopsis": anime["node"].get("synopsis", ""),  # Synopsis (if available)
            "genres": ", ".join([g["name"] for g in anime["node"].get("genres", [])]),  # Genres as a comma-separated string
            "mean": anime["node"].get("mean", 0),  # Average score, defaulting to 0 if missing
            "media_type": anime["node"].get("media_type", "unknown"),  # Type (TV, movie, OVA, etc.)
            "status": anime["node"].get("status", "unknown")  # Status (e.g., finished_airing, airing)
        }
        for anime in anime_list
    ])
    # Merge with existing data if provided
    if existing_df is not None:
        new_data = pd.concat([existing_df, new_data], ignore_index=True)
    return(new_data)


    


In [26]:
def compute_similarity(df):
    vectorizer = TfidfVectorizer(stop_words="english")
    tfidf_matrix = vectorizer.fit_transform(df["synopsis"])
    similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return similarity_matrix

In [27]:
def recommend_anime(user_favorites, df, similarity_matrix):
    recommended = set()
    for anime_title in user_favorites:
        if anime_title in df["title"].values:
            idx = df[df["title"].str.contains(anime_title)].index[0]
            similar_indices = similarity_matrix[idx].argsort()[-6:-1][::-1]
            recommended.update(df.iloc[similar_indices]["title"].values)
    return list(recommended)

In [28]:
#if __name__ == "__main__":
#    df = None 
    #for i in range(0, 21525, 100):
#    for i in range(0, 1000, 100):
#        anime_list = fetch_anime_list(i)
#        df = preprocess_data(anime_list,df)
#    df = combine_copies(df)
#    similarity_matrix = compute_similarity(df)
    
#    user_favorites = ['Overlord', 'Tensei shitara Slime Datta Ken']#'That Time I Got Reincarnated as a Slime']
    
#    recommendations = recommend_anime(user_favorites, df, similarity_matrix)
#    print("Recommended Anime:", recommendations)

In [29]:
df = None 
for i in range(0, 21525, 100):
#for i in range(0, 1000, 100):
    anime_list = fetch_anime_list(i)
    df = preprocess_data(anime_list,df)
df = combine_copies(df)
similarity_matrix = compute_similarity(df)

Error 504: <!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<HTML><HEAD><META HTTP-EQUIV="Content-Type" CONTENT="text/html; charset=iso-8859-1">
<TITLE>ERROR: The request could not be satisfied</TITLE>
</HEAD><BODY>
<H1>504 Gateway Timeout ERROR</H1>
<H2>The request could not be satisfied.</H2>
<HR noshade size="1px">
We can't connect to the server for this app or website at this time. There might be too much traffic or a configuration error. Try again later, or contact the app or website owner.
<BR clear="all">
If you provide content to customers through CloudFront, you can find steps to troubleshoot and help prevent this error by reviewing the CloudFront documentation.
<BR clear="all">
<HR noshade size="1px">
<PRE>
Generated by cloudfront (CloudFront) HTTP3 Server
Request ID: IrDp7RQlNNVoPo9lu-6XNGRqx3JxHZ_pylwoBXgBQ7R7tX7fUOkw7A&#x3D;&#x3D;
</PRE>
<ADDRESS>
</ADDRESS>
</BODY></HTML>
Error 504: <!DOCTYPE HTML PUBLIC "-//W3C//DTD H

In [30]:
def get_anime_titles():
    return df["title"].tolist()


In [31]:
def recommend(user_favorites):
    recommendations = recommend_anime(user_favorites, df, similarity_matrix)
    return "\n".join(recommendations)


In [32]:
def add_to_list(selected_anime, user_list):
    if selected_anime and selected_anime not in user_list:
        user_list.append(selected_anime)
    return user_list

In [33]:
def clear_list():
    return []

In [34]:
with gr.Blocks() as demo:
    gr.Markdown("# Anime Recommendation System")
    
    anime_dropdown = gr.Dropdown(get_anime_titles(), label="Select an anime to add to your list")
    user_list = gr.State([])  # Holds the list of selected anime
    anime_list_display = gr.Textbox(label="Your Anime List", interactive=False)
    
    add_button = gr.Button("Add to List")
    clear_button = gr.Button("Clear List")
    
    recommend_button = gr.Button("Get Recommendations")
    recommendations_output = gr.Textbox(label="Recommended Anime")
    
    add_button.click(add_to_list, inputs=[anime_dropdown, user_list], outputs=user_list)
    add_button.click(lambda x: "\n".join(x), inputs=user_list, outputs=anime_list_display)
    
    clear_button.click(clear_list, outputs=user_list)
    clear_button.click(lambda: "", outputs=anime_list_display)
    
    recommend_button.click(recommend, inputs=user_list, outputs=recommendations_output)

In [35]:
if __name__ == "__main__":
    demo.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [36]:
print(json.dumps(anime_list, indent=2))  # Check structure


[
  {
    "node": {
      "id": 44156,
      "title": "Sai Er Hao Movie 4: Sheng Mo Zhi Zhan",
      "main_picture": {
        "medium": "https://cdn.myanimelist.net/images/anime/1377/117987.jpg",
        "large": "https://cdn.myanimelist.net/images/anime/1377/117987l.jpg"
      },
      "alternative_titles": {
        "synonyms": [
          "Seer Movie 4: Trinity War"
        ],
        "en": "",
        "ja": "\u8d5b\u5c14\u53f7\u5927\u7535\u5f714\uff1a\u5723\u9b54\u4e4b\u6218"
      },
      "synopsis": "",
      "genres": [
        {
          "id": 15,
          "name": "Kids"
        }
      ],
      "rank": 21501,
      "popularity": 23018,
      "media_type": "movie",
      "status": "finished_airing"
    },
    "ranking": {
      "rank": 21501
    }
  },
  {
    "node": {
      "id": 44157,
      "title": "Sai Er Hao Movie 5: Leishen Jue Qi",
      "main_picture": {
        "medium": "https://cdn.myanimelist.net/images/anime/1326/117989.webp",
        "large": "https://cdn.my